<a href="https://colab.research.google.com/github/KashyapCKotak/AI-ML-experiments/blob/master/Stock1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#==============================================================================
# %% calc inputs and outputs
#==============================================================================
###helper functions
#return
ret = lambda x: np.log(x/x.shift(1))
#10-step and 100-step moving average
ma10 =  lambda x: pd.rolling_mean( x, 10, min_periods=1 )
ma100 =  lambda x: pd.rolling_mean( x, 100, min_periods=1 )
#10-step and 100-step moving maximum
max10 =  lambda x: pd.rolling_max( x, 10, min_periods=1 )
max100 =  lambda x: pd.rolling_max( x, 100, min_periods=1 )
#10-step and 100-step moving minimum
min10 =  lambda x: pd.rolling_min( x, 10, min_periods=1 )
min100 =  lambda x: pd.rolling_min( x, 100, min_periods=1 )
#this list will contain names of all inputs for convenience
factors = []
#split the data by date - because when computing moving averages, etc - we cannot roll between days
grp = df.groupby( 'DATE' )

### bidasksize factors
df['bidasksize'] = ( df['BIDSIZE'] + df['ASKSIZE'] )
df['bidasksize_notional'] = ( df['BIDSIZE'] * df['BIDPRICE'] + df['ASKSIZE'] * df['ASKPRICE'] )
df['bidasksize_imbalance'] = ( df['BIDSIZE'] - df['ASKSIZE'] ) 
df['bidasksize_imbalance_log'] = np.log( df['BIDSIZE'] / df['ASKSIZE'] ) 
df['bidasksize_imbalance_norm'] = ( df['BIDSIZE'] - df['ASKSIZE'] ) / ( df['BIDSIZE'] + df['ASKSIZE'] )
df['bidasksize_ret'] = grp['bidasksize'].apply( ret ) 
df['asksize_ret'] = grp['ASKSIZE'].apply( ret ) 
df['bidsize_ret'] = grp['BIDSIZE'].apply( ret ) 
namesBidAsk = ['bidasksize', 'bidasksize_notional', 'bidasksize_imbalance', 'bidasksize_imbalance_log', 'bidasksize_imbalance_norm', 'bidasksize_ret', 'asksize_ret', 'bidsize_ret' ]
factors += namesBidAsk
#for the above factors - calculate MA10, MA100 and diff MA1-MA10, MA10-MA100
for name in namesBidAsk:
    df[ name + '_MA10' ] = grp[ name ].apply( ma10 )
    df[ name + '_MA100' ] = grp[ name ].apply( ma100 )
    df[ name + '_MA10_dev' ] = df[ name ] - df[ name + '_MA10' ]
    df[ name + '_MA100_dev' ] = df[ name + '_MA10' ] - df[ name + '_MA100' ]

    factors += [ name + '_MA10', name + '_MA100',  name + '_MA10_dev', name + '_MA100_dev' ]

### spread factors
df[ 'spread' ] = ( df['ASKPRICE'] - df['BIDPRICE']).round(9) 
df[ 'spread_MA10' ] = grp[ 'spread' ].apply( ma10 ) 
df[ 'spread_MA100' ] = grp[ 'spread' ].apply( ma100 )
df[ 'spread_MA10_dev' ] = np.log( df[ 'spread' ] / df[ 'spread_MA10' ] )
df[ 'spread_MA100_dev' ] = np.log( df[ 'spread_MA10' ] / df[ 'spread_MA100' ] )
namesSpread = [ 'spread', 'spread_MA10', 'spread_MA100', 'spread_MA10_dev', 'spread_MA100_dev' ]
factors += namesSpread

### weighted mid factors
df['mid'] = ( df['BIDPRICE'] + df['ASKPRICE'] ) / 2
df['mid_MA10'] = grp[ 'mid' ].apply( ma10 )
df['mid_MA100'] = grp[ 'mid' ].apply( ma100 )
df['mid_weighted'] = ( df['BIDSIZE'] * df['ASKPRICE'] + df['ASKSIZE'] * df['BIDPRICE'] ) / ( df['BIDSIZE'] + df['ASKSIZE'] )
df['mid_weighted_MA10'] = grp[ 'mid_weighted' ].apply( ma10 )
df['mid_weighted_MA100'] = grp[ 'mid_weighted' ].apply( ma100 )

df['mid_weighted_dev'] = np.log( df['mid_weighted'] / df['mid'] )
df['mid_weighted_MA10_dev'] = np.log( df['mid_weighted_MA10'] / df['mid_MA10'] )
df['mid_weighted_MA100_dev'] = np.log( df['mid_weighted_MA100'] / df['mid_MA100'] )

namesWeighted = ['mid_weighted_dev', 'mid_weighted_MA10_dev', 'mid_weighted_MA100_dev']
factors += namesWeighted

### trades factors
df['buysell'] = df['isBuy'] * 2 - 1
df['buy'] = df['buysell'] > 0
df['sell'] = df['buysell'] < 0
df['size_signed'] = df['buysell']*df['SIZE']

df['buysell_MA10'] = grp['buysell'].apply( ma10 )
df['buysell_MA100'] = grp['buysell'].apply( ma100 )
df['buy_MA10'] = grp['buy'].apply( ma10 )
df['buy_MA100'] = grp['buy'].apply( ma100 )
df['sell_MA10'] = grp['sell'].apply( ma10 )
df['sell_MA100'] = grp['sell'].apply( ma100 )
df['size_buy_MA10'] = (df['buy']*df['SIZE']).groupby(df['DATE']).apply( ma10 )
df['size_buy_MA100'] = (df['buy']*df['SIZE']).groupby(df['DATE']).apply( ma100 )
df['size_sell_MA10'] = (df['sell']*df['SIZE']).groupby(df['DATE']).apply( ma10 )
df['size_sell_MA100'] = (df['sell']*df['SIZE']).groupby(df['DATE']).apply( ma100 )
df['size_signed_MA10'] = grp['size_signed'].apply( ma10 )
df['size_signed_MA100'] = grp['size_signed'].apply( ma100 )
df['size_MA10'] = grp['SIZE'].apply( ma10 )
df['size_MA100'] = grp['SIZE'].apply( ma100 )
df['size_signed_MA10_norm'] = df['size_signed_MA10'] / df['size_MA10']
df['size_signed_MA100_norm'] = df['size_signed_MA100'] / df['size_MA100']

namesTrades = ['buysell_MA10', 'buysell_MA100', 'buy_MA10', 'buy_MA100', 'sell_MA10', 'sell_MA100',
    'size_buy_MA10', 'size_buy_MA100', 'size_sell_MA10', 'size_sell_MA100', 'size_signed_MA10', 'size_signed_MA100', 
    'size_MA10', 'size_MA100', 'size_signed_MA10_norm', 'size_signed_MA100_norm' ]

factors += namesTrades

### price dynamics factors
df['mid_ma10'] = grp['mid'].apply( ma10 )
df['mid_ma100'] = grp['mid'].apply( ma100 )
df['mid_max10'] = grp['mid'].apply( max10 )
df['mid_max100'] = grp['mid'].apply( max100 )
df['mid_min10'] = grp['mid'].apply( min10 )
df['mid_min100'] = grp['mid'].apply( min100 )

df['mid_max10_dev'] = np.log( df['mid_max10'] / df['mid'] )
df['mid_max100_dev'] = np.log( df['mid_max100'] / df['mid'] )
df['mid_min10_dev'] = np.log( df['mid_min10'] / df['mid'] )
df['mid_min100_dev'] = np.log( df['mid_min100'] / df['mid'] )
df['mid_maxmin10_dev'] = np.log( df['mid_max10'] / df['mid_min10'] )
df['mid_maxmin100_dev'] = np.log( df['mid_max100'] / df['mid_min100'] )
df['mid_ma10_dev'] = np.log( df['mid'] / df['mid_ma10'] )
df['mid_ma100_dev'] = np.log( df['mid_ma10'] / df['mid_ma100'] )
df['mid_ret'] = grp['mid'].apply( ret )
df['mid_ret_MA10'] = grp['mid_ret'].apply( ma10 )
df['mid_ret_MA100'] = grp['mid_ret'].apply( ma100 )

namesMid = ['mid_max10_dev', 'mid_max100_dev', 'mid_min10_dev', 'mid_min100_dev', 'mid_maxmin10_dev', 'mid_maxmin100_dev',
            'mid_ma10_dev', 'mid_ma100_dev', 'mid_ret', 'mid_ret_MA10', 'mid_ret_MA100']
factors += namesMid


In [0]:
### calculate outputs 
yname = 'updown'
horizon = 10
nameMidNext = 'mid_next%s_MA' % horizon
#moving average mid of _next_ 10 ticks
df[ nameMidNext ] = grp['mid'].apply( lambda x: pd.rolling_mean( x, horizon ) ).shift( -horizon )
#difference between average of next 10 ticks and the current mid
df[ nameMidNext + '_diff' ] = np.log( df[ 'mid_next%s_MA' % horizon ] / df['mid'] ) *10000 #in bps
#if it above or below?
df[ yname ] = np.sign( df[ nameMidNext + '_diff' ] )
#do some cleaning, strictly need this - because of numerical noise in nameMidNext + '_diff' variable
ind = df[ nameMidNext + '_diff' ].abs() < 0.1 #if difference is <0.1bps
df[ yname ][ ind ] = 0 #set output to 0

## add lagged output as one of factors
ynamePrev = 'updown_prev_nz'
df['updown_prev'] = grp[ 'updown' ].shift( horizon ) #last _observed_ y-value
df[ ynamePrev ] = df['updown_prev'].copy()
df[ ynamePrev ][ df[ 'updown_prev_nz' ]==0 ] = np.nan
df[ ynamePrev ] = grp['updown_prev_nz'].fillna( method='ffill' )
factors += [ 'updown_prev', 'updown_prev_nz' ]


In [0]:
# transform skewed names 
#TODO: plot histograms below find out for which input factors 
#   the histograms are striongly right skewed and put their names here to un-skew
#   (note that taking logarithm of 0 will give -inf, so for some variables you 
#   may get a lot of missing values - better not include such variables)
#...........................
namesSkewed = pd.Series([ ])
namesUnSkewed = 'log_' + namesSkewed
df[ namesUnSkewed ] = np.log(df[ namesSkewed ])
factors = np.setdiff1d( factors, namesSkewed ).tolist()
factors += namesUnSkewed.tolist()


In [0]:
#TODO: suggest your own addittional factor(s) (at least 1) and compute them here
#       explain their meaning in the doc


In [0]:
#==============================================================================
# %% check distribs and timeseries of inputs
# TODO: explain what you can conclude from these plots
#==============================================================================

grp = df.groupby('DATE')
date = min(grp.groups.keys() )
df1 = grp.get_group( date )
plotInputsHistTimeseries( df1, factors, path )        


In [0]:
#==============================================================================
# %% filter out null and zero y's
#==============================================================================
y = df[ yname ]
print "Null values %s, up %s, down %s, 0s %s" % ( y.isnull().sum(), (y > 0).sum(), (y < 0).sum(), (y == 0).sum() )
print "removing nulls and 0's"
df = df[ y.notnull() & (y!=0)  ]

# check number of NaN's and Inf's in the input variables
print "number of NaN values:\n", df[ factors ].isnull().sum().sort_values( inplace = False )
print "number of Inf values:\n", np.isinf( df[ factors ] ).sum().sort_values( inplace = False )



In [0]:
#==============================================================================
# %% prepare fit/test dataframes
#==============================================================================
dates = df['DATE'].unique()
grp = df.groupby('DATE')
dffit = grp.get_group( dates[0] )
dftest = grp.get_group( dates[1] )

#sample equal number of objects for fitting
grp = dffit.groupby( yname )
nsample = min( grp.apply(len) )
dffit = grp.apply( lambda x: x.sample( nsample ) )

In [0]:
#==============================================================================
# %% selected factors
#==============================================================================
#TODO: do variable selection (see code below, not in this cell), 
#       select top best few factors and put them here
#      your selected factors should be diverse in meaning
#       list the selected variables in the document
#    meanwhile, before you do that - use all the factors
if False:
    factorsSel = [

    ]
else:
    factorsSel = factors

In [0]:
#==============================================================================
# %% prepare fit and test data numpy arrays with selected factors 
#TODO: why do we need to use numpy arrays here? explain in the doc.
#==============================================================================
yfit = dffit[ yname ].values
xfit = dffit[ factorsSel ].values
#remove records where x or y is NaN or inf
ind = np.isnan(yfit) | np.isinf(yfit) | np.isnan(xfit).any( axis=1 ) | np.isinf(xfit).any( axis=1 )
xfit, yfit = xfit[ ~ind, :], yfit[~ind]

ytest = dftest[ yname ].values
ytestPrev = dftest[ ynamePrev ].values
xtest = dftest[ factorsSel ].values
#remove records where x or y is NaN or inf
ind = np.isnan( ytest ) | np.isinf( ytest ) | np.isnan( ytestPrev ) | np.isinf( ytestPrev ) | np.isnan( xtest ).any( axis=1 ) | np.isinf( xtest ).any( axis=1 )
xtest, ytest, ytestPrev = xtest[ ~ind, :], ytest[~ind], ytestPrev[~ind]


In [0]:
#==============================================================================
# %% create a classfier object
#==============================================================================
#this is a flag indicating which classifier we currently use
clf_type = 'forest' #'forest', 'logreg'

clfs = {}

#TODO: create random forest classifier

clfs[ 'forest' ] = clf

#TODO: create logistic regression classifier

clfs[ 'logreg' ] = clf 

if clf_type in clfs:
    clf = clfs[ clf_type ]
else:
    clf = None
    raise ValueError()

In [0]:
#==============================================================================
# %% feature selection with random forest (may take time)
#==============================================================================
#TODO: Do feature selection with random forest.
#   Analyze the selected features, their inruition. 
#   How feature selection is affected by the # of trees in the forest






In [0]:
#==============================================================================
# %% plot features importances
#==============================================================================
# TODO: plot a bar-plot with feature importances 

In [0]:
#==============================================================================
# %% features selection with lasso-logistic regression 
#==============================================================================
from sklearn.preprocessing import scale
logreg = sklearn.linear_model.LogisticRegression( penalty='l1', solver='liblinear' )
#TODO: explain why we do scale here
xfitscale = scale(xfit)
#TODO: choose a reasonable C here and fit lasso regression
#   explain how result depends on C and how you select C
#   compare with features selected by forest
C = 1
logreg.set_params(C=C)
logreg.fit( xfitscale, yfit )


In [0]:
#==============================================================================
# %% fit the classifier and test on test data
#==============================================================================
for clf_type, clf in clfs.iteritems():
    print '='*80, '\n', clf_type
    clf.fit( xfit, yfit )
        
    yfitpred = clf.predict( xfit )
    print '-'*80, '\n', "Train sample perfomance"
    #TODO: here print the Accuracy, Precision and Recall of classification for each class separately

    ytestpred = clf.predict( xtest )
    
    print '-'*80, '\n', "Test sample perfomance"
    #TODO: here print the Accuracy, Precision and Recall of classification for each class separately

#TODO: explain the fitting/testing results - is there overfitting? 
#   if so, what do you do to reduce overfitting?

In [0]:
#==============================================================================
# %% print logreg params
#==============================================================================
clf = clfs['logreg']
#TODO: print coefficients of logistic regression
#       analyze them - argue that they make sense (or not)

In [0]:
#==============================================================================
# %% do cross validation of the classifier
#==============================================================================
for clf_type, clf in clfs.iteritems():
    print '='*80, '\n', clf_type
    #TODO: do cross-validation of each classifier (use sklearn function for this) and print the results
    #       discuss them - are they good or bad?

In [0]:
#==============================================================================
# %% test random and baseline on a test sample
#==============================================================================

# test random preditions on the test data
#TODO: generate random predictions for test data and check their accuracy
# explain why you gor this accuracy



#test baseline model on the test data
#TODO: use flat predictions for test data (you computed them when computing inputs/outputs)
# and check their accuracy
# explain why you gor this accuracy


#test INVERTED baseline model on the test data
#TODO: use inverted flat predictions for test data (you computed them when computing inputs/outputs)
# and check their accuracy
# explain why you gor this accuracy


In [0]:
#==============================================================================
# %% for pairwise features - decision surfaces: 
#==============================================================================
from itertools import combinations
path2d = path + "plots/2d/"
if not os.path.exists( path2d ): os.makedirs( path2d )
#TODO: select 3 factors with different meaning and put them here
#    plot the below plots and  explain the meaning
factorsToDraw = [

        
 ]

for clf_type, clf in clfs.iteritems():
    print '='*80, '\n', clf_type

    for name1, name2 in combinations(factorsToDraw, 2): 
        print name1, name2
        plotSurface( dffit, name1, name2, yname, clf, path2d, '%s_%s_%s.png' % ( clf_type, name1, name2 ))


In [0]:
#==============================================================================
# %% overfitting example
#==============================================================================
clf = deepcopy( clfs[ 'forest' ])
#TODO: select 2 factors and generate the below example of overfitting, 
# discuss - why you see the difference in plots and what it means
factor1 = ''
factor2 = ''
for min_samples_leaf in [200, 50, 5]: 
    clf.min_samples_leaf = min_samples_leaf
    clf.min_weight_fraction_leaf = 0.0
    plotSurface( dffit, factor1 , factor2, yname, clf, path2d, 'overfit_%s.png' % min_samples_leaf )

In [0]:
#==============================================================================
# %% calc train/test sample error vs complexity 
#==============================================================================
clf = deepcopy( clfs[ 'forest' ])
clf = skl.tree.DecisionTreeClassifier()
accs_train, accs_test = [], []
complexities = range(10000, 1000, -100) + range(1000, 100, -100) + range(100, 10, -10) + range(10, 1, -1)

for complexity in complexities: 
    print complexity
    clf.min_samples_leaf = complexity
    clf.min_weight_fraction_leaf = 0.0
    #TODO: fit the forest classifier and test predictions on test data

    #TODO: memorize the accuracy on training and test data into the arrays

accs_train = pd.Series( accs_train, index=complexities )
accs_test = pd.Series( accs_test, index=complexities )

In [0]:
#==============================================================================
# %% plot train/test sample error vs complexity 
#==============================================================================
#TODO: plot how the train/test _error rate_ depend on complexity of the model
# discuss why it has the shape that it has.
#   what is the optimal complexity based on this plot?

In [0]:
#==============================================================================
# %% ROC curves: 
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#example-model-selection-plot-roc-py
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve
#==============================================================================
#TODO: plot the ROC curves for both classifiers (one overlayed on top of other), 
# discuss what you see, which model is better?
pathcurves = path + "plots/curves/"
if not os.path.exists( pathcurves ): os.makedirs( pathcurves )

#plot ROC curves for our models
for clf_type, clf in clfs.iteritems():  
    #...

#plot curve for random guessing
#TODO also plot the ROC for random predictions (you should generate random probabilities to do that)


In [0]:
#==============================================================================
# %% precision recall curves: 
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html#sklearn.metrics.precision_recall_curve
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#example-model-selection-plot-precision-recall-py
#==============================================================================
#TODO: plot the ROC curves for both classifiers (one overlayed on top of other), 
#discuss what you see, which model is better?
pathcurves = path + "plots/curves/"
if not os.path.exists( pathcurves ): os.makedirs( pathcurves )

#plot curves for our models
for clf_type, clf in clfs.iteritems():
    #...
    
#plot curve for random guessing
#TODO also plot the ROC for random predictions (you should generate random probabilities to do that)
